In [ ]:
# ==========================================
# Cell 1: 경로 설정 및 토큰 활성화
# ==========================================

from pathlib import Path
import os, sys
import json

# ---- 프로젝트 루트 ----
PROJECT_ROOT = Path("/workspace/baseball_pipeline_final")
os.chdir(PROJECT_ROOT)

if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

print("PROJECT_ROOT:", PROJECT_ROOT)

# ---- 데이터 디렉토리 ----
from src import DATA_DIR, FAISS_DIR, FISH_ROOT

INPUT_VIDEO_DIR = DATA_DIR / "input_videos"
STT_RAW_DIR = DATA_DIR / "stt_raw"
STT_SEG_DIR = DATA_DIR / "stt_segments"
LLM_OUT_DIR = DATA_DIR / "llm_outputs"
TTS_AUDIO_DIR = DATA_DIR / "tts_audio"
OUTPUT_VIDEO_DIR = DATA_DIR / "output_videos"
AUDIO_ROOT = DATA_DIR / "audio_separator"
FRAMES_ROOT = DATA_DIR / "frames"
SRC_ROOT = PROJECT_ROOT / "src"

if str(SRC_ROOT) not in sys.path:
    sys.path.append(str(SRC_ROOT))

# 디렉토리 생성
for d in (DATA_DIR, INPUT_VIDEO_DIR, STT_RAW_DIR, STT_SEG_DIR, 
          LLM_OUT_DIR, TTS_AUDIO_DIR, OUTPUT_VIDEO_DIR, AUDIO_ROOT, 
          FRAMES_ROOT, FAISS_DIR, SRC_ROOT):
    d.mkdir(parents=True, exist_ok=True)

print("\n✅ 디렉토리 생성 완료")

# ---- API 토큰 설정 ----
CLOVA_INVOKE_URL = ""
CLOVA_SECRET_KEY = ""

HF_TOKEN = ""
# OPENAI_API_KEY = "sk-proj-..."  # 필요시 입력

if HF_TOKEN and "xxx" not in HF_TOKEN:
    os.environ["HF_TOKEN"] = HF_TOKEN
    os.environ["HUGGINGFACE_HUB_TOKEN"] = HF_TOKEN
    os.environ["HUGGING_FACE_HUB_TOKEN"] = HF_TOKEN

# if OPENAI_API_KEY:
#     os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

print("✅ API 토큰 설정 완료\n")

PROJECT_ROOT: /workspace/baseball_pipeline_final

✅ 디렉토리 생성 완료
✅ API 토큰 설정 완료



In [2]:
from src.google_mp4_download import download_gdrive_video

# ====== 여기에 구글 드라이브 링크 입력 ======
gdrive_url = "https://drive.google.com/file/d/1SyQ47qpTjsE3MqEWAqNnlgWUrlfD14gC/view?usp=sharing"
VIDEO_NAME = "SSG_삼성_10_14_2025_준플레이오프_4차전.mp4"

local_video_path = download_gdrive_video(gdrive_url, dest_name=VIDEO_NAME)
video_stem = Path(VIDEO_NAME).stem

print(f"\n✅ 영상 다운로드 완료")
print(f"  video_stem: {video_stem}")
print(f"  경로: {local_video_path}\n")

[GDRIVE] file_id: 1SyQ47qpTjsE3MqEWAqNnlgWUrlfD14gC
[GDRIVE] url    : https://drive.google.com/uc?id=1SyQ47qpTjsE3MqEWAqNnlgWUrlfD14gC
[GDRIVE] output : /workspace/baseball_pipeline_final/data/input_videos/SSG_삼성_10_14_2025_준플레이오프_4차전.mp4


Downloading...
From (original): https://drive.google.com/uc?id=1SyQ47qpTjsE3MqEWAqNnlgWUrlfD14gC
From (redirected): https://drive.google.com/uc?id=1SyQ47qpTjsE3MqEWAqNnlgWUrlfD14gC&confirm=t&uuid=0273ce8d-b032-41f8-8a59-c0c1232b4f4d
To: /workspace/baseball_pipeline_final/data/input_videos/SSG_삼성_10_14_2025_준플레이오프_4차전.mp4
100%|██████████| 494M/494M [00:12<00:00, 39.9MB/s] 

[GDRIVE] 다운로드 완료: /workspace/baseball_pipeline_final/data/input_videos/SSG_삼성_10_14_2025_준플레이오프_4차전.mp4

✅ 영상 다운로드 완료
  video_stem: SSG_삼성_10_14_2025_준플레이오프_4차전
  경로: /workspace/baseball_pipeline_final/data/input_videos/SSG_삼성_10_14_2025_준플레이오프_4차전.mp4



In [ ]:
# ==========================================
# Cell 3: audio_separator 음성 분리
# ==========================================
from src.audio_separator import separate_audio_sota

track_dict = separate_audio_sota(
    video_path=str(local_video_path),
    output_dir=str(AUDIO_ROOT),
    device="cuda"
)

vocals_path = track_dict["vocals"]
no_vocals_path = track_dict["no_vocals"]

print(f"\n✅ 음성 분리 완료! (SOTA Performance)")
print(f"  - 해설(Vocals)    : {vocals_path}")
print(f"  - 현장음(No Vocals): {no_vocals_path}\n")


In [ ]:
# ==========================================
# Cell 4: STT (Clova Speech API)
# ==========================================
from pathlib import Path
import json

from src.stt_pipeline import run_stt_pipeline
from src.stt_event_splitter import stt_json_to_event_sets

print(f"[STT] Clova STT 시작")
print(f"  입력: {vocals_path}")

STT_KEYWORD_XLSX = PROJECT_ROOT / "stt.xlsx"
if STT_KEYWORD_XLSX.exists():
    xlsx_path = STT_KEYWORD_XLSX
    use_domain = False
    print("  키워드: stt.xlsx 사용 (엑셀 부스팅)")
else:
    xlsx_path = None
    use_domain = True
    print("  키워드: 도메인 부스팅만 사용")

# ---- STT 파이프라인 실행 ----
timeline_json_path = run_stt_pipeline(
    audio_path=vocals_path,
    invoke_url=CLOVA_INVOKE_URL,
    secret_key=CLOVA_SECRET_KEY,
    stt_raw_dir=STT_RAW_DIR,
    stt_seg_dir=STT_SEG_DIR,
    xlsx_keywords_path=xlsx_path,
    use_domain_boostings=use_domain,
    speaker_count_min=2,
    speaker_count_max=3,
    save_raw_json=True,
    pause_thresh_ms=50000,  # 필요시 조절
)

print(f"\n✅ STT 완료!")
print(f"timeline.json : {timeline_json_path}\n")

[STT] Clova STT 시작
  입력: /workspace/baseball_pipeline_final/data/demucs/outputs/htdemucs/SSG_삼성_10_14_2025_준플레이오프_4차전/vocals.wav
  키워드: stt.xlsx 사용 (엑셀 부스팅)
[STT_PIPELINE] Clova STT 요청 시작: /workspace/baseball_pipeline_final/data/demucs/outputs/htdemucs/SSG_삼성_10_14_2025_준플레이오프_4차전/vocals.wav
[STT_PIPELINE] raw JSON 저장 -> /workspace/baseball_pipeline_final/data/stt_raw/vocals.clova_raw.json
[STT_PIPELINE] timeline JSON 저장 -> /workspace/baseball_pipeline_final/data/stt_segments/vocals.timeline.json

✅ STT 완료!
timeline.json : /workspace/baseball_pipeline_final/data/stt_segments/vocals.timeline.json



In [ ]:
# ==========================================
# Cell 5: STT 데이터 전처리 (이벤트 세트)
# ==========================================
from src.stt_event_splitter import stt_json_to_event_sets

timeline_json_stem = timeline_json_path.stem
# 같은 디렉터리에 *_output.json 으로 저장
json_after_split_path = timeline_json_path.with_name(f"{timeline_json_stem}_set_split.json")

# 입력 JSON 로드
with timeline_json_path.open("r", encoding="utf-8") as f:
    stt_json = json.load(f)

# 이벤트 세트 변환
event_sets = stt_json_to_event_sets(
    stt_json,
    caster_gap=10.0,   # 필요시 튜닝
    silence_gap=2.0,  # 필요시 튜닝
)

print(f"이벤트 세트 개수: {len(event_sets)}")

# 출력 JSON 저장
with json_after_split_path.open("w", encoding="utf-8") as f:
    json.dump(event_sets, f, ensure_ascii=False, indent=2)

print(f"저장 완료: {json_after_split_path.resolve()}")

이벤트 세트 개수: 63
저장 완료: /workspace/baseball_pipeline_final/data/stt_segments/vocals.timeline_set_split.json


In [ ]:
# ==========================================
# Cell 6: 영상 이미지 추출
# ==========================================

import json

from src.image_extraction import capture_frames_for_sets

# 세트 json 로드
with json_after_split_path.open("r", encoding="utf-8") as f:
    sets = json.load(f)   # 리스트 형태여야 함

print(f"세트 개수: {len(sets)}")

# 5) 세트의 set_start_sec 기준으로 이미지 추출
results = capture_frames_for_sets(
    video_path=local_video_path,
    sets=sets,
    output_dir=FRAMES_ROOT
)

# 6) 요약 출력
success_count = sum(1 for r in results if r["success"])
print(f"성공적으로 저장된 이미지 수: {success_count}")


세트 개수: 63
[INFO] 파일: /workspace/baseball_pipeline_final/data/input_videos/SSG_삼성_10_14_2025_준플레이오프_4차전.mp4
[INFO] FPS = 59.94005994005994, 총 프레임 수 = 56486, 길이 ≈ 942.375초
[DEBUG] set_id=vocals-1, ts=30.780s, frame_idx=1845, ret=True
[INFO] 세트 vocals-1: 30.780s 프레임 저장 → /workspace/baseball_pipeline_final/data/frames/vocals-1.jpg
[DEBUG] set_id=vocals-2, ts=38.374s, frame_idx=2300, ret=True
[INFO] 세트 vocals-2: 38.374s 프레임 저장 → /workspace/baseball_pipeline_final/data/frames/vocals-2.jpg
[DEBUG] set_id=vocals-3, ts=76.600s, frame_idx=4591, ret=True
[INFO] 세트 vocals-3: 76.600s 프레임 저장 → /workspace/baseball_pipeline_final/data/frames/vocals-3.jpg
[DEBUG] set_id=vocals-4, ts=84.960s, frame_idx=5093, ret=True
[INFO] 세트 vocals-4: 84.960s 프레임 저장 → /workspace/baseball_pipeline_final/data/frames/vocals-4.jpg
[DEBUG] set_id=vocals-5, ts=98.960s, frame_idx=5932, ret=True
[INFO] 세트 vocals-5: 98.960s 프레임 저장 → /workspace/baseball_pipeline_final/data/frames/vocals-5.jpg
[DEBUG] set_id=vocals-6, ts=107.082

[h264 @ 0x19939440] mmco: unref short failure
[h264 @ 0x19939440] mmco: unref short failure


[DEBUG] set_id=vocals-56, ts=775.080s, frame_idx=46458, ret=True
[INFO] 세트 vocals-56: 775.080s 프레임 저장 → /workspace/baseball_pipeline_final/data/frames/vocals-56.jpg
[DEBUG] set_id=vocals-57, ts=831.730s, frame_idx=49854, ret=True
[INFO] 세트 vocals-57: 831.730s 프레임 저장 → /workspace/baseball_pipeline_final/data/frames/vocals-57.jpg
[DEBUG] set_id=vocals-58, ts=855.740s, frame_idx=51293, ret=True
[INFO] 세트 vocals-58: 855.740s 프레임 저장 → /workspace/baseball_pipeline_final/data/frames/vocals-58.jpg


[h264 @ 0x19939440] mmco: unref short failure
[h264 @ 0x19939440] mmco: unref short failure


[DEBUG] set_id=vocals-59, ts=864.180s, frame_idx=51799, ret=True
[INFO] 세트 vocals-59: 864.180s 프레임 저장 → /workspace/baseball_pipeline_final/data/frames/vocals-59.jpg
[DEBUG] set_id=vocals-60, ts=878.740s, frame_idx=52672, ret=True
[INFO] 세트 vocals-60: 878.740s 프레임 저장 → /workspace/baseball_pipeline_final/data/frames/vocals-60.jpg
[DEBUG] set_id=vocals-61, ts=881.300s, frame_idx=52825, ret=True
[INFO] 세트 vocals-61: 881.300s 프레임 저장 → /workspace/baseball_pipeline_final/data/frames/vocals-61.jpg
[DEBUG] set_id=vocals-62, ts=897.160s, frame_idx=53776, ret=True
[INFO] 세트 vocals-62: 897.160s 프레임 저장 → /workspace/baseball_pipeline_final/data/frames/vocals-62.jpg
[DEBUG] set_id=vocals-63, ts=921.329s, frame_idx=55225, ret=True
[INFO] 세트 vocals-63: 921.329s 프레임 저장 → /workspace/baseball_pipeline_final/data/frames/vocals-63.jpg
[DONE] 세트 기반 프레임 캡처 완료.
성공적으로 저장된 이미지 수: 63


In [ ]:
# ==========================================
# Cell 7: VLM 스코어보드 추출 (최적화 버전)
# ==========================================

from pathlib import Path

from src.vlm_scoreboard import (
    load_scoreboard_model_and_processor,
    attach_scoreboard_to_sets,
)

# ==========================
# 모델 / 프로세서 로드
# ==========================

vlm_model, vlm_processor = load_scoreboard_model_and_processor()


# ==========================
# scoreboard 파이프라인 실행
# ==========================
json_after_split_stem = json_after_split_path.stem
# 같은 디렉터리에 *_output.json 으로 저장
scoreboard_json_path = json_after_split_path.with_name(f"{json_after_split_stem}_scoreboard.json")

updated_sets = attach_scoreboard_to_sets(
    json_after_split_path=json_after_split_path,
    output_json_path=scoreboard_json_path,
    frames_root=FRAMES_ROOT,
    video_path=local_video_path,
    model=vlm_model,
    processor=vlm_processor,
    retry_if_all_null=False,   # all-null이면 +2초 재시도
    retry_offset_sec=2.0,
)

print(f"총 세트 수: {len(updated_sets)}")
print(f"저장 완료: {scoreboard_json_path.resolve()}")

# 일부만 눈으로 확인
for s in updated_sets[:5]:
    print("=" * 80)
    print("set_id:", s["set_id"])
    print("set_start_sec:", s["set_start_sec"])
    print("scoreboard:", s["scoreboard"])

compute_dtype = torch.bfloat16


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

모델 / 프로세서 로드 완료
[INFO] 세트 개수: 63
[INFO] 파일: /workspace/baseball_pipeline_final/data/input_videos/SSG_삼성_10_14_2025_준플레이오프_4차전.mp4
[INFO] FPS = 59.94005994005994, 총 프레임 수 = 56486, 길이 ≈ 942.375초

[SET] set_id=vocals-1, set_start_sec=30.780


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


[INFO] 1차 scoreboard: {'원정팀': None, '홈팀': None, '원정팀 점수': None, '홈팀 점수': None, '이닝': None, '이닝 상황': None, '볼': None, '스트라이크': None, '아웃': None, '주자': None, '투수 이름': None, '투구 수': None, '타자 이름': None, '타자 타순': None, '타자 경기 기록': None}
[INFO] 1차 결과가 all-null → +2초 위치 재시도
[DEBUG] capture ts=32.780s, frame_idx=1965, ret=True
[INFO] 프레임 저장: /workspace/baseball_pipeline_final/data/frames/vocals-1_retry.jpg
[INFO] 2차 scoreboard: {'원정팀': None, '홈팀': None, '원정팀 점수': None, '홈팀 점수': None, '이닝': None, '이닝 상황': None, '볼': None, '스트라이크': None, '아웃': None, '주자': None, '투수 이름': None, '투구 수': None, '타자 이름': None, '타자 타순': None, '타자 경기 기록': None}

[SET] set_id=vocals-2, set_start_sec=38.374
[INFO] 1차 scoreboard: {'원정팀': None, '홈팀': None, '원정팀 점수': None, '홈팀 점수': None, '이닝': None, '이닝 상황': None, '볼': None, '스트라이크': None, '아웃': None, '주자': None, '투수 이름': None, '투구 수': None, '타자 이름': None, '타자 타순': None, '타자 경기 기록': None}
[INFO] 1차 결과가 all-null → +2초 위치 재시도
[DEBUG] capture ts=40.374s, frame_idx=2420, ret=Tr

[h264 @ 0x1dd61e80] mmco: unref short failure
[h264 @ 0x1dd61e80] mmco: unref short failure


[DEBUG] capture ts=413.890s, frame_idx=24809, ret=True
[INFO] 프레임 저장: /workspace/baseball_pipeline_final/data/frames/vocals-35_retry.jpg
[INFO] 2차 scoreboard: {'원정팀': 'SSG', '홈팀': '삼성', '원정팀 점수': 0, '홈팀 점수': 1, '이닝': 5, '이닝 상황': '말', '볼': None, '스트라이크': None, '아웃': None, '주자': None, '투수 이름': None, '투구 수': None, '타자 이름': None, '타자 타순': None, '타자 경기 기록': None}

[SET] set_id=vocals-36, set_start_sec=424.890
[INFO] 1차 scoreboard: {'원정팀': 'SSG', '홈팀': '삼성', '원정팀 점수': 0, '홈팀 점수': 1, '이닝': 5, '이닝 상황': '말', '볼': 1, '스트라이크': 0, '아웃': 1, '주자': {'1루': False, '2루': False, '3루': False}, '투수 이름': '김광현', '투구 수': None, '타자 이름': '전병우', '타자 타순': 9, '타자 경기 기록': '0타수 0안타'}

[SET] set_id=vocals-37, set_start_sec=449.060
[INFO] 1차 scoreboard: {'원정팀': None, '홈팀': None, '원정팀 점수': None, '홈팀 점수': None, '이닝': None, '이닝 상황': None, '볼': None, '스트라이크': None, '아웃': None, '주자': None, '투수 이름': None, '투구 수': None, '타자 이름': None, '타자 타순': None, '타자 경기 기록': None}
[INFO] 1차 결과가 all-null → +2초 위치 재시도
[DEBUG] capture ts=451

In [ ]:
from pathlib import Path
import json

from pakchanho_commentary_generator import (
    load_pakchanho_model,
    generate_analyst_for_all_sets,
)

# Pakchanho LLM 적용 후 저장할 경로
json_llm_output_path = LLM_OUT_DIR / "vocals_timeline_set_split_scoreboard_pakchanho.json"

# 1) 모델 로드
model, tokenizer = load_pakchanho_model(
    base_model_name="kakaocorp/kanana-1.5-8b-instruct-2505",
    lora_model_id="SeHee8546/kanana-1.5-8b-pakchanho-lora-v2",
    load_in_4bit=True,
)

# 2) 세트별 analyst_text 재생성
game_title = "2025 KBO 준플레이오프 4차전 삼성 vs SSG"  # 경기 정보는 상황에 맞게 바꾸면 됨

result_sets = generate_analyst_for_all_sets(
    json_in_path=scoreboard_json_path,
    json_out_path=json_llm_output_path,
    model=model,
    tokenizer=tokenizer,
    game_title=game_title,
    temperature=0.7,
    top_p=0.9,
    repetition_penalty=1.1,
    no_repeat_ngram_size=3,
    base_max_new_tokens=512,
)

print(f"총 세트 수: {len(result_sets)}")
print(f"저장 완료: {json_llm_output_path.resolve()}")

# 3) 샘플 몇 개 확인
for row in result_sets[:5]:
    print("=" * 80)
    print("set_id:", row["set_id"])
    print("caster_text:", row["caster_text"])
    print("analyst_text:", row["analyst_text"])

[INFO] 자동 생성된 경기 설명: 2025 준플레이오프 4차전 SSG vs 삼성

[DIALOGUE] 캐스터-해설 대화 생성 시작
  경기: 2025 준플레이오프 4차전 SSG vs 삼성
  입력: /workspace/skn17_final_runpod_code/baseball_pipeline/data/llm_outputs/SSG_삼성_10_14_2025_준플레이오프_4차전.utter_vlm_scoreboard.csv
[DIALOGUE] 캐스터-해설 대화 생성
입력: /workspace/skn17_final_runpod_code/baseball_pipeline/data/llm_outputs/SSG_삼성_10_14_2025_준플레이오프_4차전.utter_vlm_scoreboard.csv
출력: /workspace/skn17_final_runpod_code/baseball_pipeline/data/llm_outputs/SSG_삼성_10_14_2025_준플레이오프_4차전.dialogue_commentary.csv
해설 최소 구간: 5.0초
[DIALOGUE] 총 88개 moment 로드
[DIALOGUE] 211개 주요 이벤트 감지
[DIALOGUE] loading base model on cuda ...


`torch_dtype` is deprecated! Use `dtype` instead!
[03:00:52] INFO: We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).
Loading checkpoint shards: 100%|██████████| 4/4 [00:04<00:00,  1.17s/it]


[DIALOGUE] loading LoRA: SeHee8546/kanana-1.5-8b-pakchanho-lora
[DIALOGUE] model ready.

[1/88] t=0:00:07 (dur=22.7s, utter_id=0)
  캐스터 (10.2s): 이호성이 이 리드를 지켜냅니다. 끝나 이런 경기가 있습니다. 오중간을 갈라놓습니다....
  해설   (12.5s): 아직 한 방만 나오면 됩니다. 이제는 어떻게든지 아웃 카운트 하나라도 잡고, 점수를 내줬으면 다시 우리 쪽으로 넘겨주는 그런 ...

[2/88] t=0:00:30 (dur=25.8s, utter_id=1)
  캐스터 (47.1s): 1번 타자 박성한 2번 에레디아, 3번 타자 최정 4번 한유성, 5번 고명 등 수많은 한국 시리즈 무대를 올라섰던 김광현 선수...
  해설   (-21.3s): 김광현이라는 투수는 포크볼과 체인지업 위주로 변화구 피칭을 하는 투수로 유명했습니다. 그래서 상대 타자가 공을 기다리는 타이밍...

[3/88] t=0:00:55 (dur=12.5s, utter_id=2)
  캐스터 (3.8s): 태안 사람 태안으로 길이 보전하...
  해설   (8.7s): 네, 자, 어, 어제도 한번 말씀 드렸었는데요. 자, 어, 원래 이닝마다의 제일 중요한 순간이 바로 이 첫 번째 타자를 상대를...

[4/88] t=0:01:15 (dur=5.7s, utter_id=3)
  캐스터 (5.8s): 에레디아가 후라도 상대 홈런이 있습니다. 신선진...
  해설   (-0.0s): 네. 자, 우선 투수와 타자는 서로 눈빛이 통했다고 할까요? 그렇지는 않았겠죠. 왜냐하면은 갑자기 이렇게 투수가 바뀌었으니까요...

[5/88] t=0:01:23 (dur=13.0s, utter_id=4)
  캐스터 (13.6s): 지금 에레디아 최정, 한유섬의 스윙 삼진 연속 탈삼진 1회를 삼자 범퇴로 시작을 하고 있는 쿠라도 선수입니다....
  해설   (-0.6s): 쿠라도라는 투수가 이번 포

In [ ]:
# ==========================================
# JSON 기반 TTS 전체 파이프라인
# ==========================================

from pathlib import Path
import pandas as pd

from src.json_tts_pipeline import run_full_tts_pipeline_from_json


# 이 영상에 대한 정보들
video_stem = local_video_path.stem

# LLM을 통과한 최종 세트 JSON (지금 올려준 파일)
# 현재는 이름이 "vocals_timeline_set_split_scoreboard_pakchanho (4).json" 이니까
# 1) 파일명을 위 규칙으로 바꾸거나
# 2) 그냥 정확한 이름을 직접 Path 로 넣어도 됩니다.
# json_sets_path = DATA_DIR / "llm_outputs" / "vocals_timeline_set_split_scoreboard_pakchanho (4).json"

# Fish-Speech API 설정
FISH_API_URL = "http://127.0.0.1:8080/v1/tts"

CASTER_REF_WAVS = [DATA_DIR / "tts_refs" / "caster_prompt_1.wav"]
ANALYST_REF_WAVS = [DATA_DIR / "tts_refs" / "analyst_pakchanho_prompt_1.wav"]

print(f"[TTS] JSON 기반 TTS 파이프라인 시작")
print(f"  JSON 세트 파일: {json_llm_output_path}")
print(f"  원본 영상: {local_video_path}")
print(f"  캐스터 참조: {CASTER_REF_WAVS}")
print(f"  해설 참조: {ANALYST_REF_WAVS}")

try:
    final_tts_wav, aligned_csv, tts_csv_with_paths = run_full_tts_pipeline_from_json(
        json_sets_path=json_llm_output_path,
        video_path=local_video_path,
        caster_ref_wavs=CASTER_REF_WAVS,
        analyst_ref_wavs=ANALYST_REF_WAVS,
        fish_api_url=FISH_API_URL,
        # 아래 파라미터들은 기존 셋업 그대로 사용 (필요하면 튜닝 가능)
        min_text_chars=2,
        merge_same_role=True,
        merge_gap_thresh_sec=0.25,
        merge_short_thresh_sec=1.0,
        min_gap_sec=0.02,
        caster_extra_ratio=0.2,
        analyst_extra_ratio=2.0,
        max_analyst_expand_sec=7.0,
        analyst_priority_min_overlap_sec=0.5,
        min_gap_ms=60,
        tail_margin_ms=80,
        caster_max_speedup=1.3,
        analyst_max_speedup=1.8,
    )

    print("\n✅ JSON → TTS → 정렬 → WSOLA 전체 완료!")
    print(f"  - TTS CSV(with paths): {tts_csv_with_paths}")
    print(f"  - 정렬 CSV: {aligned_csv}")
    print(f"  - 최종 TTS 타임라인 wav: {final_tts_wav}\n")

    # 통계 출력 (선택)
    tts_df = pd.read_csv(tts_csv_with_paths)
    print("📊 TTS 통계:")
    print(f"  - 총 발화 수: {len(tts_df)}")
    print(f"  - TTS 성공: {tts_df['tts_wav_path'].notna().sum()}개")
    print(f"  - TTS 실패: {tts_df['tts_wav_path'].isna().sum()}개")

except Exception as e:
    print(f"\n❌ JSON 기반 TTS 파이프라인 실패: {e}")
    import traceback
    traceback.print_exc()
    raise


[ADAPTER] 데이터 형식 변환 시작
  video_stem: SSG_삼성_10_14_2025_준플레이오프_4차전
[ADAPTER] 데이터 형식 변환 시작
  입력: /workspace/skn17_final_runpod_code/baseball_pipeline/data/llm_outputs/SSG_삼성_10_14_2025_준플레이오프_4차전.dialogue_commentary.csv
  video_stem: SSG_삼성_10_14_2025_준플레이오프_4차전
[ADAPTER] 변환 완료:
  출력: /workspace/skn17_final_runpod_code/baseball_pipeline/data/llm_outputs/SSG_삼성_10_14_2025_준플레이오프_4차전.tts_phrases.from_dialogue.csv
  총 moments: 88개
  총 utterances: 111개
    - 캐스터: 88개
    - 해설: 23개

✅ 형식 변환 완료: /workspace/skn17_final_runpod_code/baseball_pipeline/data/llm_outputs/SSG_삼성_10_14_2025_준플레이오프_4차전.tts_phrases.from_dialogue.csv

📊 변환 결과:
  - 총 utterances: 111개
  - source_video: SSG_삼성_10_14_2025_준플레이오프_4차전.mp4
  - 캐스터: 88개
  - 해설: 23개



In [ ]:
# ==========================================
# Cell 12: 최종 영상 인코딩
# ==========================================

import subprocess

OUTPUT_VIDEO_DIR = DATA_DIR / "final_videos"
OUTPUT_VIDEO_DIR.mkdir(parents=True, exist_ok=True)

# Demucs 등에서 만든 no_vocals wav 경로 (기존 파이프라인에 맞게 설정)
vocals_path = DATA_DIR / "demucs" / f"{video_stem}.vocals.wav"
no_vocals_path = DATA_DIR / "demucs" / f"{video_stem}.no_vocals.wav"

def merge_audio_and_encode_video(
    original_video_path: Path,
    tts_vocals_path: Path,
    bg_no_vocals_path: Path,
    output_video_path: Path,
    tts_volume: float = 1.0,
    bg_volume: float = 0.7,
) -> Path:
    """
    1) TTS vocals + 배경음(no_vocals) 믹싱
    2) 원본 비디오와 합쳐서 최종 영상 생성
    """
    output_video_path.parent.mkdir(parents=True, exist_ok=True)
    
    cmd = [
        "ffmpeg", "-y",
        "-i", str(original_video_path),
        "-i", str(tts_vocals_path),
        "-i", str(bg_no_vocals_path),
        "-filter_complex",
        f"[1:a]volume={tts_volume}[a1];"
        f"[2:a]volume={bg_volume}[a2];"
        f"[a1][a2]amix=inputs=2:duration=first[amix]",
        "-map", "0:v:0",
        "-map", "[amix]",
        "-c:v", "copy",
        "-c:a", "aac",
        "-shortest",
        str(output_video_path),
    ]
    
    print("[ENCODING] 최종 영상 인코딩 중...")
    print(" ".join(cmd))
    subprocess.run(cmd, check=True)
    
    print(f"[ENCODING] 완료: {output_video_path}")
    return output_video_path


final_video_path = OUTPUT_VIDEO_DIR / f"{video_stem}.final.mp4"

print(f"[ENCODING] 최종 영상 생성 시작")
print(f"  원본 비디오: {local_video_path}")
print(f"  TTS 음성: {final_tts_wav}")
print(f"  배경음: {no_vocals_path}")
print(f"  출력: {final_video_path}")

try:
    final_video = merge_audio_and_encode_video(
        original_video_path=local_video_path,
        tts_vocals_path=final_tts_wav,
        bg_no_vocals_path=no_vocals_path,
        output_video_path=final_video_path,
        tts_volume=1.0,
        bg_volume=0.7,
    )
    
    print("\n" + "="*80)
    print("🎉 전체 파이프라인 완료!")
    print("="*80)
    print(f"최종 영상: {final_video}")
    print(f"파일 크기: {final_video.stat().st_size / (1024**2):.2f} MB")
    print("="*80 + "\n")

except Exception as e:
    print(f"\n❌ 최종 인코딩 실패: {e}")
    import traceback
    traceback.print_exc()
    raise


[ENCODING] 최종 영상 생성 시작
  원본 비디오: /workspace/skn17_final_runpod_code/baseball_pipeline/data/input_videos/SSG_삼성_10_14_2025_준플레이오프_4차전.mp4
  TTS 음성: /workspace/skn17_final_runpod_code/baseball_pipeline/data/tts_audio/SSG_삼성_10_14_2025_준플레이오프_4차전/SSG_삼성_10_14_2025_준플레이오프_4차전.tts_timeline.wav
  배경음: /workspace/skn17_final_runpod_code/baseball_pipeline/data/demucs/outputs/htdemucs/SSG_삼성_10_14_2025_준플레이오프_4차전/no_vocals.wav
  출력: /workspace/skn17_final_runpod_code/baseball_pipeline/data/output_videos/SSG_삼성_10_14_2025_준플레이오프_4차전.final.mp4
[ENCODING] 최종 영상 인코딩 중...
ffmpeg -y -i /workspace/skn17_final_runpod_code/baseball_pipeline/data/input_videos/SSG_삼성_10_14_2025_준플레이오프_4차전.mp4 -i /workspace/skn17_final_runpod_code/baseball_pipeline/data/tts_audio/SSG_삼성_10_14_2025_준플레이오프_4차전/SSG_삼성_10_14_2025_준플레이오프_4차전.tts_timeline.wav -i /workspace/skn17_final_runpod_code/baseball_pipeline/data/demucs/outputs/htdemucs/SSG_삼성_10_14_2025_준플레이오프_4차전/no_vocals.wav -filter_complex [1:a]volume=1.0[a1];[2:a]vol

ffmpeg version 6.1.1-3ubuntu5 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13 (Ubuntu 13.2.0-23ubuntu3)
  configuration: --prefix=/usr --extra-version=3ubuntu5 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --disable-omx --enable-gnutls --enable-libaom --enable-libass --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgme --enable-libgsm --enable-libharfbuzz --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-libzimg --ena

[ENCODING] 완료: /workspace/skn17_final_runpod_code/baseball_pipeline/data/output_videos/SSG_삼성_10_14_2025_준플레이오프_4차전.final.mp4

🎉 전체 파이프라인 완료!
최종 영상: /workspace/skn17_final_runpod_code/baseball_pipeline/data/output_videos/SSG_삼성_10_14_2025_준플레이오프_4차전.final.mp4
파일 크기: 464.52 MB


📊 전체 데이터 흐름 검증
✅ 원본 영상               : /workspace/skn17_final_runpod_code/baseball_pipeline/data/input_videos/SSG_삼성_10_14_2025_준플레이오프_4차전.mp4
   크기: 471.06 MB
✅ Demucs vocals       : /workspace/skn17_final_runpod_code/baseball_pipeline/data/demucs/outputs/htdemucs/SSG_삼성_10_14_2025_준플레이오프_4차전/vocals.wav
   크기: 158.54 MB
✅ Demucs no_vocals    : /workspace/skn17_final_runpod_code/baseball_pipeline/data/demucs/outputs/htdemucs/SSG_삼성_10_14_2025_준플레이오프_4차전/no_vocals.wav
   크기: 158.54 MB
✅ STT raw JSON        : /workspace/skn17_final_runpod_code/baseball_pipeline/data/stt_raw/vocals.clova_raw.json
   크기: 0.18 MB
✅ STT utterances      : /workspace/skn17_final_runpod_code/baseball_pipeline/data/stt_segments/SSG_삼성_10_

[out#0/mp4 @ 0x63f4643e2700] video:466198kB audio:8022kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.304856%
size=  475665kB time=00:15:42.42 bitrate=4134.7kbits/s speed=40.1x    
[aac @ 0x63f4643e4240] Qavg: 202.753
